In [19]:
import os
from pathlib import PurePath
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
outputdir = PurePath(os.environ['DATA'], 'STEREO/L7tum/')
csvf = PurePath(outputdir, 'training_set', 'targets.csv')
df = pd.read_csv(csvf, index_col=0)
df

,frame,blob_id_init,blob_id,x,y
0,0,21,21,26.0,110.0
1,0,52,52,16.0,461.0
2,0,159,159,53.0,124.0
3,0,346,346,112.0,107.0
4,0,369,369,293.0,205.0
...,...,...,...,...,...
195,9,1172,1536,127.0,258.0
196,9,915,1874,265.0,164.0
197,9,277,27,477.0,94.0
198,9,774,607,539.0,509.0


In [17]:
df[(df['frame']==0) & (df['blob_id_init']==21)]

,frame,blob_id_init,blob_id,x,y
0,0,21,21,26.0,110.0


In [29]:
df[df['blob_id_init']==21]

,frame,blob_id_init,blob_id,x,y
0,0,21,21,26.0,110.0
20,1,21,27,26.0,112.0
40,2,21,32,24.0,123.0
60,3,21,42,24.0,137.0
80,4,21,59,24.0,138.0
100,5,21,100,23.0,143.0
120,6,21,185,23.0,148.0
140,7,21,358,23.0,156.0
160,8,21,643,23.0,161.0
180,9,21,1155,23.0,166.0


In [21]:
nframes = 10
blob_inits = df[df['frame'] == 0]['blob_id_init']
nblobs = len(blob_inits)

In [31]:
pos_array = np.zeros((nblobs, nframes, 2))
for f in range(nframes):
    for i, b in enumerate(blob_inits):
        pos_array[i, f, 0] = df[(df['frame'] == f) & (df['blob_id_init']==b)]['x']
        pos_array[i, f, 1] = df[(df['frame'] == f) & (df['blob_id_init']==b)]['y']

print('(x,y) coordinates of the 1st blob through time')
pos_array[0, :, :]

(x,y) coordinates of the 1st blob through time


array([[ 26., 110.],
       [ 26., 112.],
       [ 24., 123.],
       [ 24., 137.],
       [ 24., 138.],
       [ 23., 143.],
       [ 23., 148.],
       [ 23., 156.],
       [ 23., 161.],
       [ 23., 166.]])

In [33]:
vel_array = pos_array[:, 1:, :] - pos_array[:, 0:-1, :]
print('(vx, vy) of the 1st blob through time')
vel_array[0, :, :]

(vx, vy) of the 1st blob through time


array([[ 0.,  2.],
       [-2., 11.],
       [ 0., 14.],
       [ 0.,  1.],
       [-1.,  5.],
       [ 0.,  5.],
       [ 0.,  8.],
       [ 0.,  5.],
       [ 0.,  5.]])

In [34]:
avg_vel = pos_array[:, -1, :] - pos_array[:, 0, :]
avg_vel

array([[-3., 56.],
       [-3., 66.],
       [ 2., 61.],
       [-1., 51.],
       [ 0., 65.],
       [-1., 48.],
       [ 0., 57.],
       [-5., 49.],
       [ 0., 66.],
       [ 0., 70.],
       [ 1., 59.],
       [-2., 33.],
       [-2., 44.],
       [ 2., 73.],
       [ 3., 53.],
       [ 0., 53.],
       [-1., 70.],
       [-2., 35.],
       [-1., 57.],
       [-3., 60.]])

### Get physical coordinates

In [36]:
DSUN_OBS= 1.433675e6  # Solar diameter in km
RSUN = DSUN_OBS/2  # Solar radius in km
DT = 5*60 # 5 minutes time interval between each frame
CDELT2  = 0.0141875720451562 # solar radius / px
VUNIT = CDELT2 * RSUN/DT 

In [42]:
avg_vel_km = avg_vel*VUNIT/9
avg_vel_km

array([[-11.30020408, 210.93714291],
       [-11.30020408, 248.60448986],
       [  7.53346939, 229.77081638],
       [ -3.76673469, 192.10346943],
       [  0.        , 244.83775516],
       [ -3.76673469, 180.80326535],
       [  0.        , 214.7038776 ],
       [-18.83367347, 184.57000004],
       [  0.        , 248.60448986],
       [  0.        , 263.67142863],
       [  3.76673469, 222.23734699],
       [ -7.53346939, 124.30224493],
       [ -7.53346939, 165.73632657],
       [  7.53346939, 274.97163272],
       [ 11.30020408, 199.63693882],
       [  0.        , 199.63693882],
       [ -3.76673469, 263.67142863],
       [ -7.53346939, 131.83571432],
       [ -3.76673469, 214.7038776 ],
       [-11.30020408, 226.00408169]])

In [45]:
avg_vel_km.mean(axis=0)

array([ -3.01338776, 212.06716332])